In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from evaluate import load

tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")
model.eval()




/home/james/CodingProjects/Local_level_model_explanations/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-20 11:16:35.121842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 11:16:35.226416: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64:/home/james/Downloads/TensorRT-8.5.1.7/lib
2023-01-20 11:16:35.226430: I tenso

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [3]:
references = ["hello world", "hello world"]
candidates = ["hi universe", "bye world"]

with torch.no_grad():
  scores = model(**tokenizer(references, candidates, return_tensors='pt'))[0].squeeze()

print(scores) # tensor([1.0327, 0.2055])

tensor([1.0327, 0.2055])


In [6]:
references = [f'Given that the likelihoods of class C1 and class C2 are 36.38% and 63.62%, respectively, the case under consideration is labelled as C2 by the model. The top relevant features for this prediction decision are F9, F12, F16, and F6, while the least relevant features include F5 and F1. From the analysis performed, the values of F9, F12, and F16 are shown to have strong positive support for C2, while F6 and F13 are the top negative attributes, pulling the prediction towards C1. However, the combined magnitude of their impacts on the model is less than that of the positive input features mentioned above, hence the moderately strong push towards labelling the case as C2. The values of some features show a low influence on C2 prediction and these are F2, F10, F5, and F1.']
candidates = ['The model predicted class C2 for the given case with a confidence level of 63.62%. However, there is a 36.38% chance that it could be C1 instead. The above prediction decision is mainly due to the influence of features such as F9, F6, and F12. On the other hand, the least important features are F5 and  F1. In terms of the direction of influence or contribution of each input feature, only F3 has a negative impact, shifting the prediction verdict in favour of an alternative label. Finally, on the contrary, all the remaining features have positive contributions, increasing the likelihood of labelling the provided example as "C2". All things considered, we can conclude that the uncertainty associated with the classification decision here may be attributed to some combination of negative features coupled with strong positive attributions from F11, F7, And F4.']

with torch.no_grad():
  scores = model(**tokenizer(references, candidates, return_tensors='pt'))[0].squeeze()

print(scores) # tensor([1.0327, 0.2055])

tensor(-0.0436)


In [7]:
import evaluate

In [9]:
rouge = evaluate.load('rouge')
# predictions = ["hello there", "general kenobi"]
# references = ["hello there", "general kenobi"]
results = rouge.compute(predictions=candidates,
                         references=references)
print(results)

{'rouge1': 0.5, 'rouge2': 0.11510791366906475, 'rougeL': 0.2571428571428572, 'rougeLsum': 0.2571428571428572}


In [6]:
references = [f'Given that the likelihoods of class C1 and class C2 are 36.38% and 63.62%, respectively, the case under consideration is labelled as C2 by the model. The top relevant features for this prediction decision are F9, F12, F16, and F6, while the least relevant features include F5 and F1. From the analysis performed, the values of F9, F12, and F16 are shown to have strong positive support for C2, while F6 and F13 are the top negative attributes, pulling the prediction towards C1. However, the combined magnitude of their impacts on the model is less than that of the positive input features mentioned above, hence the moderately strong push towards labelling the case as C2. The values of some features show a low influence on C2 prediction and these are F2, F10, F5, and F1.']
candidates = ['The model predicted class C2 for the given case with a confidence level of 63.62%. However, there is a 36.38% chance that it could be C1 instead. The above prediction decision is mainly due to the influence of features such as F9, F6, and F12. On the other hand, the least important features are F5 and  F1. In terms of the direction of influence or contribution of each input feature, only F3 has a negative impact, shifting the prediction verdict in favour of an alternative label. Finally, on the contrary, all the remaining features have positive contributions, increasing the likelihood of labelling the provided example as "C2". All things considered, we can conclude that the uncertainty associated with the classification decision here may be attributed to some combination of negative features coupled with strong positive attributions from F11, F7, And F4.']

bleurt = load("bleurt", module_type="metric", checkpoint="BLEURT-20")
results = bleurt.compute(predictions=candidates, references=references)
print(results)

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: evaluate.load('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint /home/james/.cache/huggingface/metrics/bleurt/default/downloads/extracted/d6b01862d09a8feced08a9ee5a0c887edbafd7f450ddd83f5907da1bfbbc8754/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.
{'scores': [-0.13729774951934814]}
